In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from UnarySim.sw.kernel.linear import UnaryLinear
from UnarySim.sw.bitstream.gen import RNG, SourceGen, BSGen
from UnarySim.sw.metric.metric import ProgressiveError


In [5]:
rng = "Sobol"

in_feature = 1
out_feature = 1

mode = "unipolar"
scaled = True
bias=False

fc = torch.nn.Linear(in_feature, out_feature, bias=bias).cuda()
print(fc)
if mode is "unipolar":
    fc.weight.data = torch.rand(out_feature, in_feature).mul(256).round().div(256)
    if bias is True:
        fc.bias.data = torch.rand(1, out_feature).mul(256).round().div(256)
elif mode is "bipolar":
    fc.weight.data = torch.rand(out_feature, in_feature).mul(2).sub(1).mul(256).round().div(256)
    if bias is True:
        fc.bias.data = torch.rand(1, out_feature).mul(2).sub(1).mul(256).round().div(256)
ufc = UnaryLinear(in_feature, out_feature, 256, fc.weight, fc.bias, mode=mode, scaled=scaled, bias=bias).cuda()

print("weight:")
print(fc.weight, fc.weight.size())

if bias is True:
    print("\nbias:")
    print(fc.bias, fc.bias.size())

iVec = (torch.rand(1, in_feature)*256).round()/256
oVec = fc(iVec)

print("\ninput:")
print(iVec)
print("\nexpected output:")
print(oVec)

if scaled is False:
    oVec = oVec.clamp(-1.,1.).mul(256).floor()/256
else:
    if bias is True:
        oVec = oVec.div(in_feature+1).mul(256).floor()/256
    else:
        oVec = oVec.div(in_feature).mul(256).floor()/256

print("\nexpected quantized output:")
print(oVec)

Linear(in_features=1, out_features=1, bias=False)
weight:
Parameter containing:
tensor([[0.7539]], requires_grad=True) torch.Size([1, 1])

input:
tensor([[0.7109]])

expected output:
tensor([[0.5360]], grad_fn=<MmBackward>)

expected quantized output:
tensor([[0.5352]], grad_fn=<DivBackward0>)


In [16]:
iVecSource = SourceGen(iVec, bitwidth=8, mode=mode).Gen()

iVecRNG = RNG(8, 1, rng).Out()
iVecBS = BSGen(iVecSource, iVecRNG)

iVecPP = ProgressiveError(iVec, mode=mode)
oVecPP = ProgressiveError(oVec, mode=mode)

with torch.no_grad():
    idx = torch.zeros(iVecSource.size()).type(torch.long)
    for i in range(256):
        iBS = iVecBS.Gen(idx + i)
        iVecPP.Monitor(iBS)
        oVecU = ufc(iBS)

        oVecPP.Monitor(oVecU)
    print("input error: ", min(min(iVecPP.Report())), max(max(iVecPP.Report())))
    print("output error:", min(min(oVecPP.Report())), max(max(oVecPP.Report())))
    print(oVecPP.Report())
    print(oVecPP.one_cnt)
    print(oVecPP.out_pp)



input error:  tensor(0.) tensor(0.)
output error: tensor(0.1055) tensor(0.1055)
tensor([[0.1055]])
tensor([[55.]])
tensor([[0.2148]])
